# Example profiling of `empymod`

In [1]:
import numpy as np
from line_profiler import LineProfiler
%load_ext memory_profiler

import empymod as epm

## Profiler `do_profile`

The function `do_profile` is taken from:
  - Blog : https://zapier.com/engineering/profiling-python-boss/
  - GitHub: https://github.com/zapier/profiling-python-like-a-boss/blob/master/timers.py.

License:

Copyright (c) 2013, Zapier Inc. All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

   - Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
   - Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
   - Neither the name of the Zapier Inc. nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [2]:
def do_profile(follow=[]):
    """
    Simply decorate an expensive function and optionally
    pass other functions to follow. For example:
        def get_number():
            for x in xrange(5000000):
                yield x
        @do_profile(follow=[get_number])
        def expensive_function():
            for x in get_number(5000000):
                i = x ^ x ^ x
            return 'some result!'
    """
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner

### Define Model

In [3]:
src   = [0, 0, 400]
rec   = [np.arange(1,1001)*10, np.arange(1,1001)*3, 400]
depth = [ 0, 200, 500, 600, 1100, 1300, 1800, 2300, 3000]
res   = [2e14, 4, 10, 20, 1/.2, 1/.3, 1/.4, 1/.5, 1/.6, 1/.7]
aniso  = [.5, 4, 2, 1.5, 3, .25, 2, .5, 1, 2]
epermH = [1, 80, 5, 10, 20, 3, 20, 5, 8, 10]
epermV = [1, 80, 5, 20, 10, 8, 10, 5, 6, 5]
mpermH = [1, 1, 1.0001, 1, 1, 1.00001, 1, 1, 1, 1]
mpermV = [1, 1, 1, 1.0001, 1.0001, 1, 1, 1, 1, 1]
freq = np.arange(1,21)

## Profile

Activate or adjust depending on what you want to profile

In [4]:
#@do_profile(follow=[epm.frequency, epm.fht, epm.wavenumber])
#@do_profile(follow=[epm.check_ab, epm.check_param, epm.check_spatial, epm.check_frequency])

@do_profile(follow=[epm.dlf, epm.greenfct, epm.reflections, epm.fields])
def test_function():
    out = epm.dipole(src=src, rec=rec, depth=depth, res=res, ab=11, freqtime=freq,
                     ht='FHT', htarg=epm.filters.key_401_2009(),
                     verb=1, xdirect=False, opt='spline',
                     aniso=aniso, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV)
result = test_function()

Timer unit: 1e-06 s

Total time: 0.188387 s
File: /home/dtr/anaconda3/lib/python3.6/site-packages/empymod/kernel.py
Function: greenfct at line 136

Line #      Hits         Time  Per Hit   % Time  Line Contents
   136                                           def greenfct(zsrc, zrec, lsrc, lrec, depth, etaH, etaV, zetaH, zetaV, lambd,
   137                                                        ab, xdirect, msrc, mrec, use_ne_eval):
   138                                               """Calculate Green's function for TM and TE.
   139                                           
   140                                               .. math:: \\tilde{g}^{tm}_{hh}, \\tilde{g}^{tm}_{hz},
   141                                                         \\tilde{g}^{tm}_{zh}, \\tilde{g}^{tm}_{zz},
   142                                                         \\tilde{g}^{te}_{hh}, \\tilde{g}^{te}_{zz}
   143                                           
   144                                      


Line #      Hits         Time  Per Hit   % Time  Line Contents
   413                                           def fields(depth, Rp, Rm, Gam, lrec, lsrc, zsrc, ab, TM, use_ne_eval):
   414                                               """Calculate Pu+, Pu-, Pd+, Pd-.
   415                                           
   416                                               .. math:: P^{u\pm}_s, P^{d\pm}_s, \\bar{P}^{u\pm}_s, \\bar{P}^{d\pm}_s;
   417                                                     P^{u\pm}_{s-1}, P^{u\pm}_n, \\bar{P}^{u\pm}_{s-1}, \\bar{P}^{u\pm}_n;
   418                                                     P^{d\pm}_{s+1}, P^{d\pm}_n, \\bar{P}^{d\pm}_{s+1}, \\bar{P}^{d\pm}_n
   419                                           
   420                                               This function corresponds to equations 81/82, 95/96, 103/104, A-8/A-9,
   421                                               A-24/A-25, and A-32/A-33 in [Hunziker_et_al_2015]_, and loosely to the


## Memory and other tests

In [6]:
opt = 'spline'
HT='FHT'

print('timeit')
%timeit epm.dipole(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freqtime=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)

print('\nmemit')
%memit epm.dipole(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freqtime=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)

print('\nmprun')
%mprun -f epm.greenfct epm.dipole(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freqtime=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)

print('\nprun')
%prun -l 10 epm.dipole(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freqtime=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)


timeit
90 ms ± 3.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

memit
peak memory: 95.78 MiB, increment: 2.50 MiB

mprun


prun
 

In [7]:
epm.versions('HTML')